In [74]:
import json
import ast
import pandas as pd
from urllib.parse import urlparse
import numpy as np
from urllib.parse import urlparse
import nltk
nltk.download('punkt')

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()


DATA_DIRECTORY = '../../data/reddit/Article_data_2019/'
candidates = [ "TRUMP", "SANDERS","DELANEY","WARREN","HARRIS","GILLIBRAND","O'ROURKE","KLOBUCHAR","BOOKER",
    "BUTTIGIEG","GABBARD","YANG","INSLEE","HICKENLOOPER","WILLIAMSON","DE LA FUENTE","CASTRO","BIDEN"]
candidates = set([x.lower() for x in candidates])

[nltk_data] Downloading package punkt to /Users/kippy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The following ceel is for converting old formats to the new format in pandas (as opposed to csvs). This cell can be skipped if you have a pandas pkl with article text in it already.

In [43]:
## load the data into a dataframe, spit out the row of an error if any
df2 = pd.read_pickle(DATA_DIRECTORY + 'reddit_2019_06_15.pkl')
list_df = []
i = 0
for row in open(DATA_DIRECTORY + "/article_data.csv", 'r').readlines():
    url, data = row.split(', ', 1)
    try:
        data = ast.literal_eval(data)
        list_df.append((url, data))
    except Exception as e:
        print(i, e)    
    i += 1 
df = pd.merge(df2, pd.DataFrame(list_df, columns = ['url', 'article_data']), on='url', how='left', )


In [48]:
df.to_pickle(DATA_DIRECTORY + 'reddit_2019_06_15_with_article_text.pkl')

In [111]:
PICKLE_NAME = DATA_DIRECTORY + "reddit_2019_06_15_with_article_text.pkl"
df = pd.read_pickle(PICKLE_NAME)

IsADirectoryError: [Errno 21] Is a directory: '../../data/reddit/Article_data_2019/'

In [112]:
clean_df = pd.concat( [df.drop(['article_data'],axis=1), df['article_data'].progress_apply(pd.Series)]
                      ,axis = 1)
clean_df.drop(0, axis = 1, inplace=True)


def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

clean_df = df_column_uniquify(clean_df)

In [109]:
# How many are valid?
clean_df[(~clean_df['title_1'].isnull()) & (~clean_df['text'].isnull())].shape[0], clean_df.shape[0]

(82018, 121986)

In [113]:
clean_df = clean_df[(~clean_df['title_1'].isnull()) & (~clean_df['text'].isnull())]
clean_df["title_tokenized"] = clean_df.progress_apply(lambda x: [token.lower() for token in nltk.word_tokenize(x.title)]
                                                      , axis = 1)
clean_df["text_tokenized"] = clean_df.progress_apply(lambda x: [token.lower() for token in nltk.word_tokenize(x.text)]
                                                     , axis = 1)
clean_df["domain"] = clean_df.progress_apply(lambda x: urlparse(x.url).netloc, axis = 1)
clean_df['candidate_title'] = clean_df.progress_apply(lambda x: candidates.intersection(x.title_tokenized), axis=1)
clean_df['candidate_text'] = clean_df.progress_apply(lambda x: candidates.intersection(x.text_tokenized), axis=1)

In [117]:
clean_df.memory_usage(deep=True).agg('sum')

1893446375

In [131]:
len(set(clean_df['url']))

64581

In [128]:
clean_df[clean_df['candidate_text'] == set(('castro', 'trump'))]

,id,num_comments,score,subreddit,subreddit_subscribers,title,url,authors,date_download,date_modify,...,text,title_1,title_page,title_rss,url_1,title_tokenized,text_tokenized,domain,candidate_title,candidate_text
225,absaqo,3,1,politics,4440257,Cuba's Castro: Trump Has Put U.S. On Road to C...,https://www.newsweek.com/cubas-castro-trump-ha...,[],2019-06-09 18:59:29,None,...,Former Cuban President Raul Castro said the U....,Cuba's Castro: Trump Has Put U.S. on Road to C...,NaN,NaN,https://www.newsweek.com/cubas-castro-trump-ha...,"[cuba, 's, castro, :, trump, has, put, u.s., o...","[former, cuban, president, raul, castro, said,...",www.newsweek.com,"{castro, trump}","{castro, trump}"
527,abvchi,5,1,politics,4441591,Pompeo discusses Venezuela with Brazil's new r...,https://www.reuters.com/article/us-usa-brazil-...,"[Reuters Editorial, Min Read]",2019-06-09 19:07:51,None,...,BRASILIA (Reuters) - U.S. Secretary of State M...,"Pompeo, Brazil's new government target Cuba, V...",NaN,NaN,https://www.reuters.com/article/us-usa-brazil-...,"[pompeo, discusses, venezuela, with, brazil, '...","[brasilia, (, reuters, ), -, u.s., secretary, ...",www.reuters.com,{},"{castro, trump}"
1836,adc34p,8,0,politics,4468241,Crossing the Southern Border Illegally is very...,https://www.npr.org/2017/07/25/539263390/death...,"[Joel Rose, Shaheen Ainpour]",2019-06-09 20:54:49,None,...,Death At The Southern Border An Increasing Ris...,Death At The Southern Border An Increasing Ris...,NaN,NaN,https://www.npr.org/2017/07/25/539263390/death...,"[crossing, the, southern, border, illegally, i...","[death, at, the, southern, border, an, increas...",www.npr.org,{},"{castro, trump}"
1895,add5p9,47,1,politics,4468819,Julian Castro cites 90 percent tax on rich in ...,https://www.washingtonexaminer.com/news/julian...,[],2019-06-09 20:56:18,None,...,Likely 2020 Democratic presidential candidate ...,Julian Castro cites 90 percent tax on rich in ...,NaN,NaN,https://www.washingtonexaminer.com/news/julian...,"[julian, castro, cites, 90, percent, tax, on, ...","[likely, 2020, democratic, presidential, candi...",www.washingtonexaminer.com,{castro},"{castro, trump}"
2012,adfy32,3,1,politics,4470239,Interesting read,https://www.psychologytoday.com/us/blog/the-hu...,[],2019-06-09 20:59:48,None,...,"Whether narcissism is a real disorder, as oppo...",Does Trump Suffer from Narcissistic Personalit...,NaN,NaN,https://www.psychologytoday.com/us/blog/the-hu...,"[interesting, read]","[whether, narcissism, is, a, real, disorder, ,...",www.psychologytoday.com,{},"{castro, trump}"
2699,admqyt,29,1,politics,4473400,Democrats Warn Trump Against Declaring A Natio...,https://www.buzzfeednews.com/article/johnstant...,"[John Stanton, Buzzfeed News Reporter]",2019-06-09 21:19:53,None,...,Catarina Gomez Lucas shows a picture of her 8-...,Democrat Lawmakers Say There Is No National Se...,NaN,NaN,https://www.buzzfeednews.com/article/johnstant...,"[democrats, warn, trump, against, declaring, a...","[catarina, gomez, lucas, shows, a, picture, of...",www.buzzfeednews.com,{trump},"{castro, trump}"
2716,adn1e7,1586,11,politics,4473553,News networks should not air Trump Wall speech...,https://www.usatoday.com/story/opinion/2019/01...,"[Kurt Bardella, Opinion Columnist, Published P...",2019-06-09 21:20:29,None,...,"CLOSE With no breakthrough in sight, President...",News nets shouldn't air Trump address to the n...,NaN,NaN,https://www.usatoday.com/story/opinion/2019/01...,"[news, networks, should, not, air, trump, wall...","[close, with, no, breakthrough, in, sight, ,, ...",www.usatoday.com,{trump},"{castro, trump}"
3400,ae2nse,2,1,politics,4481536,"Oligarch, friend of Trump: Who is Pavel Fuchs?",https://www.aljazeera.com/indepth/features/oli...,"[Will Jordan, About The Author]",2019-06-09 21:56:39,None,...,"""When I was young, I beat people up,"" Pavel Fu...","Oligarch, friend of Trump: Who is Pavel Fuchs?",NaN,NaN,https://www.aljazeera.com/indepth/features/oli...,"[oligarch, ,, friend, of, trump, :, who

In [118]:
candidate_counts_title = {}
for item in clean_df.apply(lambda x: tuple(x.candidate_title), axis=1):
    candidate_counts_title[item] = candidate_counts_title.get(item, 0) + 1
candidate_counts_title

{(): 44350,
 ('warren',): 1013,
 ('trump', 'warren'): 165,
 ('sanders',): 2227,
 ('trump',): 27227,
 ('castro', 'trump'): 17,
 ('trump', 'sanders'): 381,
 ('inslee',): 200,
 ('biden',): 2006,
 ('klobuchar', 'warren'): 5,
 ('klobuchar',): 138,
 ('harris',): 615,
 ('sanders', 'warren'): 83,
 ("o'rourke",): 390,
 ('trump', 'biden'): 389,
 ('booker',): 185,
 ('gabbard',): 222,
 ('castro',): 86,
 ('trump', "o'rourke"): 83,
 ('trump', 'harris'): 79,
 ('trump', 'sanders', 'biden'): 42,
 ('yang',): 275,
 ('harris', 'booker'): 13,
 ('klobuchar', 'trump'): 20,
 ('gillibrand',): 130,
 ('gabbard', 'sanders', 'warren'): 3,
 ('trump', 'gabbard'): 23,
 ('trump', 'booker'): 25,
 ('harris', 'warren'): 8,
 ('buttigieg',): 765,
 ('williamson',): 11,
 ('hickenlooper',): 29,
 ('gabbard', 'sanders'): 13,
 ('trump', 'harris', 'biden'): 4,
 ('harris', 'biden'): 22,
 ('delaney',): 30,
 ('harris', 'warren', 'biden'): 6,
 ('gillibrand', 'harris', 'booker'): 1,
 ('warren', 'harris', 'booker'): 1,
 ('harris', 'san

In [119]:
candidate_counts_text = {}
for item in clean_df.apply(lambda x: tuple(x.candidate_text), axis=1):
    candidate_counts_text[item] = candidate_counts_text.get(item, 0) + 1
candidate_counts_text

{(): 20931,
 ('sanders', 'warren'): 145,
 ('trump', 'sanders'): 3038,
 ('klobuchar', 'sanders'): 6,
 ('trump', 'sanders', 'warren', 'biden'): 222,
 ('klobuchar',
  'gillibrand',
  'warren',
  'harris',
  'trump',
  'sanders',
  'biden',
  'booker'): 12,
 ('trump', 'sanders', 'warren'): 368,
 ('sanders',): 1022,
 ('buttigieg', 'trump', 'sanders', 'biden'): 58,
 ('trump',): 40037,
 ('harris',): 460,
 ('trump', 'buttigieg'): 288,
 ('inslee', 'sanders', 'warren', 'booker'): 4,
 ('trump', 'warren'): 947,
 ('gillibrand', 'trump', 'warren'): 28,
 ('trump', 'buttigieg', 'warren', 'biden'): 26,
 ('warren',): 551,
 ('castro', 'trump'): 293,
 ('inslee', 'harris', 'warren', 'trump', 'booker'): 5,
 ('klobuchar', 'gillibrand', 'warren', 'trump', 'sanders', "o'rourke"): 7,
 ('trump', 'booker'): 111,
 ('inslee', 'harris', 'warren', 'trump', 'sanders', 'booker'): 4,
 ('klobuchar',
  'gillibrand',
  'harris',
  'warren',
  'trump',
  'sanders',
  'biden',
  "o'rourke",
  'booker'): 4,
 ('klobuchar',
  '